# 使用WAM预测donor位点
> 生科登峰1901班，吴思承
>
> U201912536

## 概述
真核生物的基因包含外显子和内含子，在转录完成后，需要进行加工，选择性地将外显子连接起来，最终得到成熟的mRNA。而研究显示，在内含子与外显子的边界上，存在保守的剪接位点。其中，我们将位于内含子左侧（5'端）的剪接位点称为供体（donor），而位于内含子右侧（3'端）的剪接位点称为受体（acceptor）。

在基因预测工作中，研究人员往往需要明确DNA中实际编码蛋白质的部分。此时便需要预测RNA上的选择性剪接方式，而作为内含子边界上的保守序列，donor位点的识别对内含子、外显子的位置、数量的确定有着巨大的用处。

预测donor位点的方法包括WMM、WAM到SVN、神经网络等，颇为丰富。本上机报告选择实现其中的SVM模型，并对其实际预测效果进行评估。

## 方法

### SVM的原理


### 依赖加载与数据挂载
与先前的WAM上机任务类似，在本次上机中，本人同样使用Python（Jupyter Notebook）实现SVM模型。代码在Google Colab的云服务上运行，数据存储在账号对应的Google Drive中。

调用的依赖库中，``os``用于读取文件，``re``用于正则表达式辨别数据中的外显子信息，``random``提供随机采样函数，``matplotlib``用于绘制图像，``tqdm``用于生成数据读取的进度条，``sklearn``用于提供现成的模型性能评估方法和SVM实现。

In [6]:
import os
import re
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, f1_score, confusion_matrix

# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 数据储存路径
training_path = '/content/drive/MyDrive/Collab Files/donor_dataset/testing'
testing_path = '/content/drive/MyDrive/Collab Files/donor_dataset/testing'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 从数据集中提取序列片段
与WAM不同，SVM不仅仅通过统计donor位点附近的碱基分布来构建打分表，而是需要同时使用阳性样本和阴性样本的序列片段来构建判别器。因此需要对训练数据中的各种序列片段进行提取。

下面的``get_sequence``函数重复使用了WAM上机任务中的代码。

In [2]:
# get_sequence：从数据集中提取需要的序列片段
def get_sequence(file_path, k1=4, k2=4, type='positive'):
    sequence_list = []

    # 逐个读取文件
    print('Handling files in \'' + file_path + '\' ...')
    for data_file in tqdm(os.listdir(file_path)):
        with open(file_path + '/' + data_file, 'r') as FILE:
            lines = FILE.readlines()
            sequence = ''.join(lines[2:]).replace('\n', '').lower()
            donor_sites = re.findall('(\d+)(?=,)', lines[1])

            # 输出阳性样本
            if type == 'positive':
                for site in donor_sites:
                    site_num = eval(site)
                    subsequence = sequence[site_num - k1 - 1:site_num + k2]

                    if (set(subsequence) | {'a', 't', 'c', 'g'}) != {'a', 't', 'c', 'g'}:
                        continue

                    sequence_list.append(subsequence)

            # 输出阴性样本
            else:
                window_size = k1 + k2 + 1
                seq_len = len(sequence)

                for i in range(seq_len - window_size + 1):
                    subsequence = sequence[i:i + window_size]

                    if (set(subsequence) | {'a', 't', 'c', 'g'}) != {'a', 't', 'c', 'g'} \
                       or str(i + k1 + 1) in donor_sites:
                        continue
                    
                    sequence_list.append(subsequence)

    return sequence_list

### 对序列进行one-hot编码
由于SVM将样本序列视为高维空间中的一个点（向量），用0~3的数字来表示某一位置上的碱基将会引入原始数据中不存在的位置关系，可能会对模型产生负面影响。因此，有必要对碱基进行one-hot编码（独热编码），将离散的碱基映射到欧氏空间中，并确保不同碱基之间的距离相同，以便后续训练。

函数``one_hot_sequence``将输入的序列处理为one-hot编码的numpy数组，输出的数组长度将会是输入序列长度的4倍。

碱基到one-hot编码的映射关系储存在字典``nucleotide_to_array``中。与WAM中的处理方法类似，表示不确定碱基的``n``、``b``等将会被直接忽略。

In [3]:
# nucleotide_to_array：提供从碱基到one-hot编码的映射关系
nucleotide_to_array = {
    'a': np.array([1, 0, 0, 0], dtype=float),
    't': np.array([0, 1, 0, 0], dtype=float),
    'c': np.array([0, 0, 1, 0], dtype=float),
    'g': np.array([0, 0, 0, 1], dtype=float)
}

print('nucleotide_to_array: ', nucleotide_to_array)

nucleotide_to_array:  {'a': array([1., 0., 0., 0.]), 't': array([0., 1., 0., 0.]), 'c': array([0., 0., 1., 0.]), 'g': array([0., 0., 0., 1.])}


In [4]:
# one_hot_sequence：将序列处理为one-hot编码的数组
def one_hot_sequence(sequence, nucleotide_to_array=nucleotide_to_array):
    one_hot_list = [nucleotide_to_array[nt] for nt in sequence]
    one_hot_array = np.array(one_hot_list).reshape(-1)
    return one_hot_array

### 构建训练集
结合前面的两个函数，按照需要构建one-hot编码完毕的数据集，用于训练。

函数``build_dataset``返回两个列表``data_list``和``data_class``，分别储存编码后的样本序列和样本的类型（是否是donor位点附近序列）。输入中的``pos_num``和``neg_num``为数据集中阳性样本和阴性样本的数量，为默认值``None``或大于样本总数时输出全部样本。

In [7]:
# build_dataset：构建训练集
def build_dataset(file_path, pos_num=None, neg_num=None, k1=4, k2=4):
    # 从训练用文件中提取所有的序列
    positive_seq = get_sequence(file_path, k1, k2, 'positive')
    negative_seq = get_sequence(file_path, k1, k2, 'negative')

    # 随机选取pos_num个阳性样本
    if pos_num != None and pos_num < len(positive_seq):
        index = random.sample(range(len(positive_seq)), pos_num)
        positive_list = [one_hot_sequence(positive_seq[i]) for i in index]
    else:
        positive_list = [one_hot_sequence(seq) for seq in positive_seq]
    
    # 随机选取neg_num个阴性样本
    if neg_num != None and neg_num < len(negative_seq):
        index = random.sample(range(len(negative_seq)), pos_num)
        negative_list = [one_hot_sequence(negative_seq[i]) for i in index]
    else:
        negative_list = [one_hot_sequence(seq) for seq in negative_seq]

    # 合并成训练集
    data_list = positive_list + negative_list
    data_class = [1] * len(positive_list) + [0] * len(negative_list)

    return data_list, data_class